In [1]:
import os
import glob
import json
import pandas as pd

In [11]:
dataF = 'G:/edu/sem2/766/proj/ML_OD_Benchmarking/data/outputs/'
origF = dataF +'5000_original_results/'
tranF = dataF + 'transformed_outputs/'
algoF = 'e2e_faster_rcnn_R-50-C4_2x/'
anntF = 'G:/edu/sem2/766/proj/ML_OD_Benchmarking/data/inputs/annotations/instances_val2017.json'

imageName = '000000076547'
imageId = int(imageName)

In [3]:
origJsons = glob.iglob(origF+algoF + '*.json')
tranJsons = glob.iglob(tranF+algoF + '*.json')
for i, js in enumerate(origJsons):
    imgs = json.load(open(js))
    for k in imgs:#array of info
        for key in k:#info is dict
            print key
    break

img_name
classes
scores
bboxes
img_name
classes
scores
bboxes
img_name
classes
scores
bboxes


In [27]:
with open(anntF) as jsonFile:
    data = json.load(jsonFile)
dt = pd.DataFrame(data['annotations'])
print len(dt.image_id.unique())
dt[dt.image_id == imageId]


4952


,area,bbox,category_id,id,image_id,iscrowd,segmentation
8865,29404.03315,"[0.0, 234.64, 196.74, 244.12]",2,125428,76547,0,"[[56.65, 235.88, 21.63, 266.78, 5.15, 307.98, ..."
9157,33899.17830,"[0.0, 130.87, 247.53, 219.55]",7,174090,76547,0,"[[2.15, 130.87, 142.06, 151.32, 130.22, 313.83..."
9258,17851.81065,"[415.11, 204.57, 172.01, 258.54]",1,188853,76547,0,"[[461.46, 290.06, 492.36, 285.94, 486.18, 298...."
9810,42.06270,"[499.88, 262.73, 3.24, 20.27]",32,296107,76547,0,"[[503.12, 265.3, 501.91, 277.46, 500.42, 283.0..."
10379,26011.63000,"[228.67, 296.45, 367.82, 174.74]",15,571957,76547,0,"[[434.7, 409.71, 395.87, 408.63, 380.76, 416.1..."
10395,5156.02745,"[449.88, 412.08, 120.34, 67.92]",15,579830,76547,0,"[[469.2, 453.48, 455.95, 435.26, 449.88, 425.8..."
10545,6922.42245,"[139.15, 283.69, 84.13, 131.59]",27,1166503,76547,0,"[[144.54, 325.75, 151.01, 373.21, 168.27, 384...."
11241,428.82520,"[276.81, 202.34, 18.62, 25.57]",10,1381942,76547,0,"[[276.81, 203.65, 278.13, 227.73, 294.3, 227.1..."
11570,1809.93785,"[385.01, 252.34, 96.34, 55.75]",73,1627609,76547,0,"[[480.11, 307.47, 429.0, 308.09, 411.65, 305.6..."
12254,2411.71875,"[496.25, 366.88, 86.25, 36.25]",62,1930335,76547,0,"[[506.25, 374.38, 496.25, 398.13, 580.0, 403.1..."
